In [23]:
from AdversarialCNN import AdversarialCNN
from sklearn.model_selection import ParameterGrid
from Utils import DataLoader
import numpy as np
from scipy.signal import resample
import pickle
import os 
import tensorflow as tf

In [2]:
# gridsearch_params = {'signal' : ['emg', 'eeg'],
#                     'class' : ['weight', 'texture'],
#                     'participants' : ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p1-9']}



#small for test
gridsearch_params = {'signal' : ['eeg'],
                    'class' : ['texture'],
                    'participants' : ['1','2','3','4','5','6','7','8','9'],
                    'batch' : ['1']}

grid = ParameterGrid(gridsearch_params)

len(grid)

9

In [3]:
for params in grid:
    print(params)

{'batch': '1', 'class': 'texture', 'participants': 1, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 2, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 3, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 4, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 5, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 6, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 7, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 8, 'signal': 'eeg'}
{'batch': '1', 'class': 'texture', 'participants': 9, 'signal': 'eeg'}


In [2]:
participantdata = DataLoader((1,9),(1,9))

done with p
done with p
done with p
done with p
done with p
done with p
done with p
done with p
done with p


In [ ]:
#creating function for leave 1 out test to make stacked participant tuple


def get_leave_one_out_train_test(participant_list,test_participant):
    '''
    NOTE: have to load participant tupled data individually before using this function
    
    test_participant: the particiant to be used as the test (int)
    '''
    
    test_split = participant_list[test_participant-1]
    
    train_split = participant_list
    

In [2]:
%%time

##implement leave 1 out test

for params in grid:
    
    participantdata.load_data_for_nn(participant=params['participants'], sigtype='both')
    
    print('\n\nMODEL: ', params, '\n-----------------------------------------------------\n\n')
    
    if(params['class'] == 'weight'):
        
        weight_pass = [1,3]
        texture_pass = [3] #sandpaper
        
    elif(params['class'] == 'texture'):
        
        weight_pass = [2]
        texture_pass = [1,3] #silk,sandpaper
        
    else:
        raise ValueError('Invalid class in passed grid dictionary')
        
    
    #make sure return_subset_for_nn works for sigtype = both
    tupleddata_paramsubset = participantdata.return_subset_for_nn(sigtype = params['signal'],
                                                              weights = weight_pass, 
                                                              textures = texture_pass)

    #SETTING UP TRAIN TEST SPLIT
    shuffled_index = np.random.permutation(range(tupleddata_paramsubset[0].shape[0]))

    tuple_tmp_element_0 = tupleddata_paramsubset[0][shuffled_index,:,:,:]
    tuple_tmp_element_1 = tupleddata_paramsubset[1][shuffled_index,:]
    tuple_tmp_element_2 = tupleddata_paramsubset[2][shuffled_index,:]

    train_test_split = int(tupleddata_paramsubset[0].shape[0] * .75)

    if(params['class']=='weight'):
        
        #dropping middle one hot encoded column of 0's
        tuple_tmp_element_1 = np.delete(tuple_tmp_element_1, 1, 1)
        
        train_tuple = (tuple_tmp_element_0[0:train_test_split,:,:,:],
                       tuple_tmp_element_1[0:train_test_split,:],
                       tuple_tmp_element_2[0:train_test_split,:])

        test_tuple = (tuple_tmp_element_0[train_test_split:,:,:,:],
                       tuple_tmp_element_1[train_test_split:,:],
                       tuple_tmp_element_2[train_test_split:,:])
        
    if(params['class']=='texture'):
        
        #dropping middle one hot encoded column of 0's
        tuple_tmp_element_2 = np.delete(tuple_tmp_element_2, 1, 1)
        
        train_tuple = (tuple_tmp_element_0[0:train_test_split,:,:,:],
                       tuple_tmp_element_2[0:train_test_split,:],
                       tuple_tmp_element_1[0:train_test_split,:])

        test_tuple = (tuple_tmp_element_0[train_test_split:,:,:,:],
                       tuple_tmp_element_2[train_test_split:,:],
                       tuple_tmp_element_1[train_test_split:,:])

    #CREATING AND RUNNING NETWORK
    if(params['signal'] == 'emg'):
        chans = 5

    elif(params['signal'] == 'eeg'):
        chans = 32
        
    elif(params['signal']=='emg+eeg'): #need to equalize samples when using both signals
        chans=37

    else:
         raise ValueError('Invalid signal type in passed grid dictionary')


    if(params['batch']=='1'):
        num_batch = 1
    
    elif(params['batch']=='max'): #set epoch to number of trials in training set
        num_batch = train_test_split
        
    else:
        raise ValueError('Invalid batch size. Tweak source code or pick 1 or max')
        
        
    net = AdversarialCNN(chans=chans, samples=tupleddata_paramsubset[0].shape[2], n_output=2, 
                         n_nuisance=3, architecture='EEGNet', adversarial=False, lam=0)  
    
    #haev it create new log for each participant
    net.train(train_tuple, test_tuple, log = 'logdir', epochs=10, batch_size=num_batch)
        
        



NameError: name 'grid' is not defined

# Converting above code into functions:
# --------------------------------------------------------

In [26]:
#main function
def gridsearch(grid, eeg_list, emg_list):
    '''
    input: param_dict. Example format: {'signal' : ['eeg','emg','both'],
                                        'class' : ['texture','weight'],
                                        'participants' : ['1','2','loo_2'],  #loo = leave one out
                                        'batch' : ['1']}
                        
           eeg_list, emg_list. obtained from: get_individiual_participant_data(9). 
             
    
    
    eeg_list, emg_list = get_individiual_participant_data(9)

    '''
    
    for params in grid:
                
        if(params['participants'][0:4] == 'loo_'):
            
            train, test = handle_leave_one_out(eeg_list, emg_list, params['signal'], 
                                               params['class'], int(params['participants'][4]) )

        else:
            
            train, test = handle_single_participant(eeg_list, emg_list, params['signal'],
                                                    params['class'], int(params['participants']) )
    
        if(params['signal'] == 'emg'): chans = 5

        elif(params['signal'] == 'eeg'): chans = 32

        elif(params['signal']=='both'): chans=37

            
        print('\n\nMODEL: ', params, '\n-----------------------------------------------------\n\n')
        log = params['signal'] + '_' + params['class'] + '_participant' + params['participants'] + '_batch' + params['batch']
        
        if not (os.path.isdir(log)):
            os.mkdir(log)
        
        net = AdversarialCNN( chans=chans, samples=train[0].shape[2], n_output=2, 
                         n_nuisance=3, architecture='EEGNet', adversarial=False, lam=0 )  
    
        net.train( train, test, log = log, epochs=30, batch_size=int(params['batch']) )
        
        model = net.acnn
        model.save(log+'_msave')
        
#         pickle_out = open(log+'_pickle','wb') 
#         pickle.dump(net, pickle_out) 
#         pickle_out.close()


        
    
#######################################################################################    
#supporting functions
def get_individiual_participant_data(num_participants):
    '''
    params: (int) total number of participants
    
    returns: list of all participant tupled data
    '''
    
    participant_data = DataLoader((1,num_participants),(1,9))
    
    participant_eeg_tuples = []
    participant_emg_tuples = []
     
    for i in range(num_participants):
        
        participant_data.load_data_for_nn(participant=i+1, sigtype='both')
        
        participant_eeg_tuple, participant_emg_tuple = participant_data.return_subset_for_nn(sigtype = 'both',
                                                                                   weights = [1,2,3], 
                                                                                    textures = [1,2,3])

        participant_eeg_tuples.append(participant_eeg_tuple)
        participant_emg_tuples.append(participant_emg_tuple)
        
        print('Loaded participant', i+1)
        
    return participant_eeg_tuples, participant_emg_tuples

def combine_eeg_emg(eegtuple, emgtuple):
    min_size = eegtuple[0].shape[2]
    emgtrials = emgtuple[0]
    resampled_trials = []
    for trial in list(np.split(emgtrials, emgtrials.shape[0])):
        resampled_channels = []
        for channel in list(np.split(trial, trial.shape[1], axis=1)):
            one_d = np.squeeze(channel)
            resampled_channel = resample(one_d, min_size)
            resampled_channel = np.reshape(resampled_channel, (-1,1))
            resampled_channels.append(resampled_channel)
        resampled_trials.append(np.hstack(resampled_channels))
    resampled_trials = np.dstack(resampled_trials)
    resampled_trials = np.reshape(resampled_trials, (resampled_trials.shape[0], resampled_trials.shape[1], resampled_trials.shape[2], 1))
    resampled_trials = np.moveaxis(np.moveaxis(resampled_trials, 0, 2), 0, 1)
    resampled_trials.shape


    combined = np.hstack((resampled_trials, eegtuple[0]))
    return (combined, eegtuple[1], eegtuple[2])

def get_min_samplesize(data):
    '''
    params: data: list of tuples
    
    return: minimum sample of list
    '''
    minimum = data[0][0].shape[2]
    for i in range(len(data)):
        if(data[i][0].shape[2] < minimum):
            minimum = data[i][0].shape[2]

    return minimum

def get_shuffled_binary_class_data_loo(train_ele_0,train_ele_1,train_ele_2,test_ele_0,test_ele_1,test_ele_2, classifier):
    '''
    description: remove third class to turn data set from multi class to binary
                    (loo = leave one out)
    params: all train/test tuple elements (6)
    
    return: elements passed in tuple form ready for fitting. shuffled and binary
    '''
    if(classifier=='texture'):
        #drop extra class
        weight_constant = np.array([0,1,0])   #we hold weight 2 constant
        texture_unwanted = np.array([0,0,1])  #we dont want texture 2
        
        train_ele_sub_0 = np.array([train_ele_0[i] for i in range(len(train_ele_0)) if np.all(train_ele_1[i] == weight_constant) 
                       and ~(np.all(train_ele_2[i] == texture_unwanted))])
        train_ele_sub_1 = np.array([train_ele_1[i] for i in range(len(train_ele_1)) if np.all(train_ele_1[i] == weight_constant) 
                       and ~(np.all(train_ele_2[i] == texture_unwanted))])
        train_ele_sub_2 = np.array([train_ele_2[i] for i in range(len(train_ele_2)) if np.all(train_ele_1[i] == weight_constant) 
                       and ~(np.all(train_ele_2[i] == texture_unwanted))])  

        test_ele_sub_0 = np.array([test_ele_0[i] for i in range(len(test_ele_0)) if np.all(test_ele_1[i] == weight_constant) 
                       and ~(np.all(test_ele_2[i] == texture_unwanted))])
        test_ele_sub_1 = np.array([test_ele_1[i] for i in range(len(test_ele_1)) if np.all(test_ele_1[i] == weight_constant) 
                       and ~(np.all(test_ele_2[i] == texture_unwanted))])  
        test_ele_sub_2 = np.array([test_ele_2[i] for i in range(len(test_ele_2)) if np.all(test_ele_1[i] == weight_constant) 
                       and ~(np.all(test_ele_2[i] == texture_unwanted))])
        
        train_ele_sub_2 = np.delete(train_ele_sub_2, 2, 1) #drop textuer column 2
        test_ele_sub_2 = np.delete(test_ele_sub_2, 2, 1) #drop textuer column 2
        
        #shuffle
        train_shuffle = np.random.permutation(range(train_ele_sub_0.shape[0]))
        test_shuffle = np.random.permutation(range(test_ele_sub_0.shape[0]))
        
        train_ele_sub_0 = train_ele_sub_0[train_shuffle,:,:,:]
        train_ele_sub_1 = train_ele_sub_1[train_shuffle,:]
        train_ele_sub_2 = train_ele_sub_2[train_shuffle,:]
        
        test_ele_sub_0 = test_ele_sub_0[test_shuffle,:,:,:]
        test_ele_sub_1 = test_ele_sub_1[test_shuffle,:]
        test_ele_sub_2 = test_ele_sub_2[test_shuffle,:]
        
        train_tuple = (train_ele_sub_0,train_ele_sub_2,train_ele_sub_1)
        test_tuple = (test_ele_sub_0,test_ele_sub_2,test_ele_sub_1)
        
    elif(classifier=='weight'):
        #drop extra class
        weight_unwanted = np.array([0,1,0])  #we dont want weight 2
        texture_constant = np.array([0,0,1]) #we hold texture 2 constant
    
        train_ele_sub_0 = np.array([train_ele_0[i] for i in range(len(train_ele_0)) if np.all(train_ele_2[i] == texture_constant) 
                       and ~(np.all(train_ele_1[i] == weight_unwanted))])
        train_ele_sub_1 = np.array([train_ele_1[i] for i in range(len(train_ele_1)) if np.all(train_ele_2[i] == texture_constant) 
                       and ~(np.all(train_ele_1[i] == weight_unwanted))])  
        train_ele_sub_2 = np.array([train_ele_2[i] for i in range(len(train_ele_2)) if np.all(train_ele_2[i] == texture_constant) 
                       and ~(np.all(train_ele_1[i] == weight_unwanted))])  

        test_ele_sub_0 = np.array([test_ele_0[i] for i in range(len(test_ele_0)) if np.all(test_ele_2[i] == texture_constant) 
                       and ~(np.all(test_ele_1[i] == weight_unwanted))])
        test_ele_sub_1 = np.array([test_ele_1[i] for i in range(len(test_ele_1)) if np.all(test_ele_2[i] == texture_constant) 
                       and ~(np.all(test_ele_1[i] == weight_unwanted))])  
        test_ele_sub_2 = np.array([test_ele_2[i] for i in range(len(test_ele_2)) if np.all(test_ele_2[i] == texture_constant) 
                       and ~(np.all(test_ele_1[i] == weight_unwanted))])      
        
        train_ele_sub_1 = np.delete(train_ele_sub_1, 1, 1) #drop textuer column 2
        test_ele_sub_1 = np.delete(test_ele_sub_1, 1, 1) #drop textuer column 2
    
        #shuffle
        train_shuffle = np.random.permutation(range(train_ele_sub_0.shape[0]))
        test_shuffle = np.random.permutation(range(test_ele_sub_0.shape[0]))
        
        train_ele_sub_0 = train_ele_sub_0[train_shuffle,:,:,:]
        train_ele_sub_1 = train_ele_sub_1[train_shuffle,:]
        train_ele_sub_2 = train_ele_sub_2[train_shuffle,:]
        
        test_ele_sub_0 = test_ele_sub_0[test_shuffle,:,:,:]
        test_ele_sub_1 = test_ele_sub_1[test_shuffle,:]
        test_ele_sub_2 = test_ele_sub_2[test_shuffle,:]
        
        train_tuple = (train_ele_sub_0,train_ele_sub_1,train_ele_sub_2)
        test_tuple = (test_ele_sub_0,test_ele_sub_1,test_ele_sub_2)
        
    return train_tuple,test_tuple

def get_shuffled_binary_class_data_single(data,classifier):
    '''
    description: remove third class to turn data set from multi class to binary
                    (loo = leave one out)
    params: single participant data in tuple form
            string classifier (texture or weight)
    
    return: elements passed in tuple form ready for fitting. shuffled and binary
    '''
    #shuffle
    shuffled_index = np.random.permutation(range(data[0].shape[0]))

    tuple_tmp_element_0 = data[0][shuffled_index,:,:,:]
    tuple_tmp_element_1 = data[1][shuffled_index,:]
    tuple_tmp_element_2 = data[2][shuffled_index,:]
    
    #train test split
    train_test_split = int(data[0].shape[0] * .75)
    
    train_ele_0 = tuple_tmp_element_0[0:train_test_split,:,:,:]
    train_ele_1 = tuple_tmp_element_1[0:train_test_split,:]
    train_ele_2 = tuple_tmp_element_2[0:train_test_split,:]
    
    test_ele_0 = tuple_tmp_element_0[train_test_split:,:,:,:]
    test_ele_1 = tuple_tmp_element_1[train_test_split:,:]
    test_ele_2 = tuple_tmp_element_2[train_test_split:,:]
    
    #dropping extra classes, forming tuple
    if(classifier=='texture'):
        weight_constant = np.array([0,1,0])   #we hold weight 2 constant
        texture_unwanted = np.array([0,0,1])  #we dont want texture 2
        
        train_ele_sub_0 = np.array([train_ele_0[i] for i in range(len(train_ele_0)) if np.all(train_ele_1[i] == weight_constant) 
                       and ~(np.all(train_ele_2[i] == texture_unwanted))])
        train_ele_sub_1 = np.array([train_ele_1[i] for i in range(len(train_ele_1)) if np.all(train_ele_1[i] == weight_constant) 
                       and ~(np.all(train_ele_2[i] == texture_unwanted))])
        train_ele_sub_2 = np.array([train_ele_2[i] for i in range(len(train_ele_2)) if np.all(train_ele_1[i] == weight_constant) 
                       and ~(np.all(train_ele_2[i] == texture_unwanted))])  

        test_ele_sub_0 = np.array([test_ele_0[i] for i in range(len(test_ele_0)) if np.all(test_ele_1[i] == weight_constant) 
                       and ~(np.all(test_ele_2[i] == texture_unwanted))])
        test_ele_sub_1 = np.array([test_ele_1[i] for i in range(len(test_ele_1)) if np.all(test_ele_1[i] == weight_constant) 
                       and ~(np.all(test_ele_2[i] == texture_unwanted))])  
        test_ele_sub_2 = np.array([test_ele_2[i] for i in range(len(test_ele_2)) if np.all(test_ele_1[i] == weight_constant) 
                       and ~(np.all(test_ele_2[i] == texture_unwanted))])
        
        train_ele_sub_2 = np.delete(train_ele_sub_2, 2, 1) #drop textuer column 2
        test_ele_sub_2 = np.delete(test_ele_sub_2, 2, 1) #drop textuer column 2
        
        train_tuple = (train_ele_sub_0,train_ele_sub_2,train_ele_sub_1)
        test_tuple = (test_ele_sub_0,test_ele_sub_2,test_ele_sub_1)
        
    elif(classifier=='weight'):
        weight_unwanted = np.array([0,1,0])  #we dont want weight 2
        texture_constant = np.array([0,0,1]) #we hold texture 2 constant
    
        train_ele_sub_0 = np.array([train_ele_0[i] for i in range(len(train_ele_0)) if np.all(train_ele_2[i] == texture_constant) 
                       and ~(np.all(train_ele_1[i] == weight_unwanted))])
        train_ele_sub_1 = np.array([train_ele_1[i] for i in range(len(train_ele_1)) if np.all(train_ele_2[i] == texture_constant) 
                       and ~(np.all(train_ele_1[i] == weight_unwanted))])  
        train_ele_sub_2 = np.array([train_ele_2[i] for i in range(len(train_ele_2)) if np.all(train_ele_2[i] == texture_constant) 
                       and ~(np.all(train_ele_1[i] == weight_unwanted))])  

        test_ele_sub_0 = np.array([test_ele_0[i] for i in range(len(test_ele_0)) if np.all(test_ele_2[i] == texture_constant) 
                       and ~(np.all(test_ele_1[i] == weight_unwanted))])
        test_ele_sub_1 = np.array([test_ele_1[i] for i in range(len(test_ele_1)) if np.all(test_ele_2[i] == texture_constant) 
                       and ~(np.all(test_ele_1[i] == weight_unwanted))])  
        test_ele_sub_2 = np.array([test_ele_2[i] for i in range(len(test_ele_2)) if np.all(test_ele_2[i] == texture_constant) 
                       and ~(np.all(test_ele_1[i] == weight_unwanted))])      
        
        train_ele_sub_1 = np.delete(train_ele_sub_1, 1, 1) #drop textuer column 2
        test_ele_sub_1 = np.delete(test_ele_sub_1, 1, 1) #drop textuer column 2
        
        train_tuple = (train_ele_sub_0,train_ele_sub_1,train_ele_sub_2)
        test_tuple = (test_ele_sub_0,test_ele_sub_1,test_ele_sub_2)
        
    return train_tuple,test_tuple
    
    
    
    
#     train_test_split = int(data[0].shape[0] * .75)
    
#     if(classifier=='weight'):
        
#         #dropping middle one hot encoded column of 0's
#         tuple_tmp_element_1 = np.delete(tuple_tmp_element_1, 1, 1)
        
#         train_tuple = (tuple_tmp_element_0[0:train_test_split,:,:,:],
#                        tuple_tmp_element_1[0:train_test_split,:],
#                        tuple_tmp_element_2[0:train_test_split,:])

#         test_tuple = (tuple_tmp_element_0[train_test_split:,:,:,:],
#                        tuple_tmp_element_1[train_test_split:,:],
#                        tuple_tmp_element_2[train_test_split:,:])
        
#     if(params['class']=='texture'):
        
#         #dropping middle one hot encoded column of 0's
#         tuple_tmp_element_2 = np.delete(tuple_tmp_element_2, 1, 1)
        
#         train_tuple = (tuple_tmp_element_0[0:train_test_split,:,:,:],
#                        tuple_tmp_element_2[0:train_test_split,:],
#                        tuple_tmp_element_1[0:train_test_split,:])

#         test_tuple = (tuple_tmp_element_0[train_test_split:,:,:,:],
#                        tuple_tmp_element_2[train_test_split:,:],
#                        tuple_tmp_element_1[train_test_split:,:])
    

def handle_leave_one_out(participant_eeg_list, participant_emg_list, signal, classifier, p_num_loo):
    '''
    Description: have to seperate leave one out as it requires different handling of
                    train test splitting and requires extracting/stacking participant data
                    
    params: participant_eeg_list: list of eeg tupled data
            participant_emg_list: list of emg tupled data
            p_num_loo: string of particiant to be used as the test
            classifier: string of class to be classified. needed for correct tuple element order (data,weight,texture)
            signal: string of signal being used for classification
            
    returns: train, test tuples ready to be passed to train function
    '''
    
    
    #setting up data list based off signal type. For eeg, emg also run sample equalization code
    if(signal == 'eeg'):
        data_list = participant_eeg_list
        minimum = get_min_samplesize(data_list)
        
    elif(signal == 'emg'):
        data_list = participant_emg_list
        minimum = get_min_samplesize(data_list)
        
    elif(signal=='both'):
        data_list = []
        for i in range(9):
            combined_data = combine_eeg_emg(participant_eeg_list[i],participant_emg_list[i])
            data_list.append(combined_data)
        minimum = get_min_samplesize(data_list)
        print('done with creating combination array')
    
    
    #TODO: add dropping of columns for different classes / SHUFFLE
    #filling train/test lists by defined data type
    #change appending system to stacking np arrays
    first = True
    for i in range(len(data_list)):
        if(signal=='eeg' or signal=='emg' or signal=='both'):
            if((i+1) != int(p_num_loo)): #if participant in list is not the one to leave and be tested on, add to train
                if(first):
                    train_ele_0 = resample(data_list[i][0],minimum,axis=2)
                    train_ele_1 = data_list[i][1]
                    train_ele_2 = data_list[i][2]
                    first = False

                else:
                    print(train_ele_0.shape,resample(data_list[i][0],minimum,axis=2).shape )
                    train_ele_0 = np.append( train_ele_0 , resample(data_list[i][0],minimum,axis=2) , 0 )
                    train_ele_1 = np.append( train_ele_1 , data_list[i][1] , 0)
                    train_ele_2 = np.append( train_ele_2 , data_list[i][2] , 0)

                print('appended participant', i+1, 'in train')

            elif((i+1) == int(p_num_loo)): #finding the one to leave out and appending to test lists
                test_ele_0 = resample(data_list[i][0],minimum,axis=2)
                test_ele_1 = data_list[i][1]
                test_ele_2 = data_list[i][2]
                print('appended participant', i+1, 'in test')
    
#     print('\n')
#     print(train_ele_0.shape,train_ele_1.shape,train_ele_2.shape)
#     print(test_ele_0.shape,test_ele_1.shape,test_ele_2.shape) 
#     print('\n')
    
    #converting to binary classification and shuffle
    train_tuple,test_tuple = get_shuffled_binary_class_data_loo(train_ele_0,train_ele_1,train_ele_2,test_ele_0,
                                                            test_ele_1,test_ele_2,classifier)
    
    print('tr texture1:', np.sum(train_tuple[1][:,0]))
    print('tr texture2:', np.sum(train_tuple[1][:,1]))
    
    print('te texture1:', np.sum(test_tuple[1][:,0]))
    print('te texture2:', np.sum(test_tuple[1][:,1])) 
    
    return train_tuple,test_tuple
        

#handle_leave_one_out(participant_eeg_list, participant_emg_list, signal, classifier, p_num_loo):
def handle_single_participant(participant_eeg_list,participant_emg_list,signal,classifier,p_num):
    
    if(signal == 'eeg'):
        data = participant_eeg_list[int(p_num)-1]
        
    elif(signal == 'emg'):
        data = participant_emg_list[int(p_num)-1]
        
    elif(signal=='both'):
        data = combine_eeg_emg(participant_eeg_list[int(p_num)-1],participant_emg_list[int(p_num)-1])

    
    print(data[0].shape)
    
    train_tuple,test_tuple = get_shuffled_binary_class_data_single(data,classifier)
    
    print('tr texture1:', np.sum(train_tuple[1][:,0]))
    print('tr texture2:', np.sum(train_tuple[1][:,1]))
    
    print('te texture1:', np.sum(test_tuple[1][:,0]))
    print('te texture2:', np.sum(test_tuple[1][:,1])) 
    
    return train_tuple,test_tuple

# TESTING

In [5]:
eeg_list, emg_list = get_individiual_participant_data(9)

done with p
done with p
done with p
done with p
done with p
done with p
done with p
done with p
done with p
Loaded participant 1
Loaded participant 2
Loaded participant 3
Loaded participant 4
Loaded participant 5
Loaded participant 6
Loaded participant 7
Loaded participant 8
Loaded participant 9


In [ ]:

param_dict = {'signal' : ['both'],
                'class' : ['weight'],
                'participants' : ['loo_3'],
                'batch' : ['20']}

grid = ParameterGrid(param_dict)


gridsearch(grid, eeg_list, emg_list)


done with creating combination array
appended participant 1 in train
(294, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 2 in train
appended participant 3 in test
(588, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 4 in train
(882, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 5 in train
(1176, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 6 in train
(1470, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 7 in train
(1764, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 8 in train
(2058, 37, 4025, 1) (294, 37, 4025, 1)
appended participant 9 in train
tr texture1: 560.0
tr texture2: 456.0
te texture1: 70.0
te texture2: 57.0


MODEL:  {'batch': '20', 'class': 'weight', 'participants': 'loo_3', 'signal': 'both'} 
-----------------------------------------------------


Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to

In [57]:
for params in grid:
    print(params)

{'batch': '1', 'class': 'weight', 'participants': '1', 'signal': 'emg'}


In [60]:
eeg_list[0][0].shape,eeg_list[1][0].shape,eeg_list[2][0].shape,eeg_list[3][0].shape,eeg_list[4][0].shape,eeg_list[5][0].shape,eeg_list[6][0].shape,eeg_list[7][0].shape,eeg_list[8][0].shape

((294, 32, 4115, 1),
 (294, 32, 4501, 1),
 (294, 32, 4061, 1),
 (294, 32, 4025, 1),
 (294, 32, 4308, 1),
 (294, 32, 4346, 1),
 (294, 32, 4755, 1),
 (294, 32, 4128, 1),
 (294, 32, 4219, 1))

In [61]:
emg_list[0][0].shape,emg_list[1][0].shape,emg_list[2][0].shape,emg_list[3][0].shape,emg_list[4][0].shape,emg_list[5][0].shape,emg_list[6][0].shape,emg_list[7][0].shape,emg_list[8][0].shape

((294, 5, 32920, 1),
 (294, 5, 36008, 1),
 (294, 5, 32488, 1),
 (294, 5, 32200, 1),
 (294, 5, 34464, 1),
 (294, 5, 34768, 1),
 (294, 5, 38040, 1),
 (294, 5, 33024, 1),
 (294, 5, 33752, 1))

In [40]:
%%time
handle_leave_one_out(eeg_list, emg_list, 'eeg', 'weight', '2')

appended participant 1 in train
appended participant 2 in test
(294, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 3 in train
(588, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 4 in train
(882, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 5 in train
(1176, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 6 in train
(1470, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 7 in train
(1764, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 8 in train
(2058, 32, 4025, 1) (294, 32, 4025, 1)
appended participant 9 in train
tr texture1: 560.0
tr texture2: 456.0
te texture1: 70.0
te texture2: 57.0
Wall time: 1min 17s


((array([[[[ 9.25835541e+01],
           [ 2.28492858e+02],
           [ 1.30285201e+02],
           ...,
           [-2.26349680e+02],
           [-1.89860507e+02],
           [-2.69433071e+02]],
  
          [[-1.35576100e+02],
           [-1.15719216e+02],
           [-9.26790935e+01],
           ...,
           [-1.49865975e+02],
           [-6.43415088e+01],
           [-8.68109798e+01]],
  
          [[ 4.87766927e+02],
           [ 5.01864813e+02],
           [ 4.76072311e+02],
           ...,
           [ 4.12660899e+02],
           [ 4.39795206e+02],
           [ 4.48372530e+02]],
  
          ...,
  
          [[-5.59720730e+01],
           [-1.77840219e+02],
           [-1.17292708e+02],
           ...,
           [ 3.84879276e+02],
           [ 4.90716748e+02],
           [ 5.69814243e+02]],
  
          [[ 3.36095851e+00],
           [-1.45436076e+02],
           [-1.05710684e+02],
           ...,
           [ 5.69020885e+02],
           [ 6.51339402e+02],
           [ 7.1

In [41]:
%%time
handle_single_participant(eeg_list, emg_list, 'eeg', 'texture', '2')

(294, 32, 4501, 1)
tr texture1: 25.0
tr texture2: 18.0
te texture1: 11.0
te texture2: 5.0
Wall time: 650 ms


((array([[[[ 1476.36873916],
           [ 1513.23507873],
           [ 1395.49429626],
           ...,
           [  676.43589598],
           [  633.81369357],
           [  487.48882681]],
  
          [[  397.88033096],
           [  356.17510316],
           [  227.79438198],
           ...,
           [  344.8368916 ],
           [  261.49483503],
           [   97.86021862]],
  
          [[  822.30416116],
           [  819.8383272 ],
           [  726.49108217],
           ...,
           [  489.64977735],
           [  432.60513486],
           [  326.96243252]],
  
          ...,
  
          [[  191.21660523],
           [   79.65804028],
           [  -72.23185088],
           ...,
           [  252.75200173],
           [   25.34141039],
           [    9.49475946]],
  
          [[  401.17535316],
           [  375.74823263],
           [  103.0028487 ],
           ...,
           [  580.10192584],
           [  245.59661516],
           [  279.96250057]],
  
          [[

In [ ]:
%%time
first = True
minimum = get_min_samplesize(eeg_list)

for i in range(len(eeg_list)):
    if(True):
        if((i+1) != int(5)): #if participant in list is not the one to leave and be tested on, add to train
            if(first):
                train_ele_0 = resample(eeg_list[i][0],minimum,axis=2)
                train_ele_1 = eeg_list[i][1]
                train_ele_2 = eeg_list[i][2]
                first = False
                print('done')
            else:
                print(train_ele_0.shape,resample(eeg_list[i][0],minimum,axis=2).shape )
                train_ele_0 = np.append( train_ele_0 , resample(eeg_list[i][0],minimum,axis=2) , 0 )
                train_ele_1 = np.append( train_ele_1 , eeg_list[i][1] , 0)
                train_ele_2 = np.append( train_ele_2 , eeg_list[i][2] , 0)

            print('appended participant', i+1, 'in train')

        elif((i+1) == int(5)): #finding the one to leave out and appending to test lists
            test_ele_0 = resample(eeg_list[i][0],minimum,axis=2)
            test_ele_1 = eeg_list[i][1]
            test_ele_2 = eeg_list[i][2]
            print('appended participant', i+1, 'in test')

In [94]:
%%time

eeg, emg = participant_data.return_subset_for_nn(sigtype = 'both',
                                                    weights = [1,2,3], 
                                                    textures = [1,2,3])

#classify texture
a = [eeg[0][i] for i in range(len(eeg[0])) if np.all(emg[1][i] == np.array([0,1,0])) and ~(np.all(emg[2][i] == np.array([0,0,1])))]
b = [eeg[1][i] for i in range(len(eeg[1])) if np.all(emg[1][i] == np.array([0,1,0])) and ~(np.all(emg[2][i] == np.array([0,0,1])))]
c = [eeg[2][i] for i in range(len(eeg[2])) if np.all(emg[1][i] == np.array([0,1,0])) and ~(np.all(emg[2][i] == np.array([0,0,1])))]

done with p
done with p
done with p
loaded participant  1 

loaded participant  2 

loaded participant  3 

Wall time: 6min 27s


In [108]:
participant_eeg_list, participant_emg_list = get_individiual_participant_data(1)

done with p
loaded participant  1 



In [110]:
from scipy.signal import resample

def combine_eeg_emg(eegtuple, emgtuple):
    min_size = eegtuple[0].shape[2]
    emgtrials = emgtuple[0]
    resampled_trials = []
    for trial in list(np.split(emgtrials, emgtrials.shape[0])):
        resampled_channels = []
        for channel in list(np.split(trial, trial.shape[1], axis=1)):
            one_d = np.squeeze(channel)
            resampled_channel = resample(one_d, min_size)
            resampled_channel = np.reshape(resampled_channel, (-1,1))
            resampled_channels.append(resampled_channel)
        resampled_trials.append(np.hstack(resampled_channels))
    resampled_trials = np.dstack(resampled_trials)
    resampled_trials = np.reshape(resampled_trials, (resampled_trials.shape[0], resampled_trials.shape[1], resampled_trials.shape[2], 1))
    resampled_trials = np.moveaxis(np.moveaxis(resampled_trials, 0, 2), 0, 1)
    resampled_trials.shape


    combined = np.hstack((resampled_trials, eegtuple[0]))
    return (combined, eegtuple[1], eegtuple[2])

combined_parti1_data = combine_eeg_emg(participant_eeg_list[0],participant_emg_list[0])

In [112]:
combined_parti1_data[0].shape

(294, 37, 4115, 1)

In [74]:
#indexing to obtain all emg data for participant 1 for classifying textures 1,3 over weight 2

a = []

for i in range(len(emg[0])):
    
    if np.all(emg[1][i] == np.array([0,1,0])) and ~(np.all(emg[2][i] == np.array([0,0,1]))):
        a.append(emg[0][i])
        
np.array(a).shape

(60, 5, 32920, 1)

# Implementation of above functions:
# --------------------------------------------------------

In [106]:
gridsearch_params = {'signal' : ['eeg'],
                    'class' : ['texture'],
                    'participants' : [1,2],
                    'batch' : ['1']}

gridsearch(gridsearch_params, 2)

done with p
done with p
loaded participant  1 

loaded participant  2 



import pickle

pickle_out = open("filename",'wb')
pickle.dump(object, pickle_out)
pickle_out.close()

#pickle_in = open("filmename",'rb')
object = pickle.load(pickle_in)